**회귀분석**\
Python을 이용하여 간단한 회귀분석 모형을 테스트해봅니다.\
우선 지난시간에 사용하였던 MLB데이터를 이용하여 간단한 회귀분석을 해 보겠습니다.\
메이저리그 팀들의 수익(Revenue)은 팀의 승률, 홈타운의 인구 수, 포스트시즌 진출여부, 월드시리즈 진출여부, 및 소속리그에 의해 결정된다고 할 때 데이터 파일로 부터 각 피쳐를 추출할 수 있습니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('reg_data/mlb_sample.csv')
print(df)

     ID  YEAR NAME        wp   w  ps  ws  AL         rev     smsa
0     1  2000   AZ  0.524691  85   0   0   0  141.191894  3278661
1     1  2001   AZ  0.567901  92   1   1   0  134.330353  3388445
2     1  2002   AZ  0.604938  98   1   0   0  153.936188  3496957
3     1  2003   AZ  0.518519  84   0   0   0  144.580612  3600163
4     1  2004   AZ  0.314815  51   0   0   0  145.447601  3723359
..   ..   ...  ...       ...  ..  ..  ..  ..         ...      ...
445  30  2010  WAS  0.425926  69   0   0   0  194.000000  5664789
446  30  2011  WAS  0.496894  80   0   0   0  193.880118  5771506
447  30  2012  WAS  0.604938  98   1   0   0  213.692866  5862594
448  30  2013  WAS  0.456790  74   0   0   0  228.392639  5949859
449  30  2014  WAS  0.592593  96   1   0   0  264.353845  6033737

[450 rows x 10 columns]


회귀분석을 위해 피쳐들의 리스트를 따로 추출하여 Matrix로 구성합니다. 종속변수는 단일벡터이기에 그냥 리스트로 추출합니다.

In [3]:
df['smsa'] = df['smsa']/1000000
columns = ['wp', 'ps', 'ws', 'AL', 'smsa']
X = df[columns].to_numpy()
y = df['rev']
print(X)
print(y)

[[0.52469136 0.         0.         0.         3.278661  ]
 [0.56790124 1.         1.         0.         3.388445  ]
 [0.60493827 1.         0.         0.         3.496957  ]
 ...
 [0.60493827 1.         0.         0.         5.862594  ]
 [0.45679012 0.         0.         0.         5.949859  ]
 [0.59259259 1.         0.         0.         6.033737  ]]
0      141.191894
1      134.330353
2      153.936188
3      144.580612
4      145.447601
          ...    
445    194.000000
446    193.880118
447    213.692866
448    228.392639
449    264.353845
Name: rev, Length: 450, dtype: float64


피쳐에 절편(intercept)을 추가합니다. 절편은 모든 원소가 1로 된 리스트로 매트릭스에 한 열로 붙여넣으면 됩니다. 어디에 붙여넣어도 상관없으나, 통상적으로 가장 왼쪽(앞쪽)에 붙여넣습니다.

In [4]:
intercept = np.ones((X.shape[0],1))
X = np.hstack((intercept,X))
print(X)

[[1.         0.52469136 0.         0.         0.         3.278661  ]
 [1.         0.56790124 1.         1.         0.         3.388445  ]
 [1.         0.60493827 1.         0.         0.         3.496957  ]
 ...
 [1.         0.60493827 1.         0.         0.         5.862594  ]
 [1.         0.45679012 0.         0.         0.         5.949859  ]
 [1.         0.59259259 1.         0.         0.         6.033737  ]]


이제 모든 피쳐와 종속변수를 만들었으니, 이를 바탕으로 회귀분석을 실시합니다.\
회귀계수의 추정치는 다음과 같이 계산할 수 있습니다.\
\
$
B = \left( X^T X \right)^{-1} X^T y
$
\
 \
파이썬에서 매트릭스를 단순히 * 를 사용해서 곱하게 되면, 이는 각 원소간 곲(inner product)을 의미합니다. 따라서 행렬간 곱하기를 하려면 numpy가 제공하는 .dot(A, B)명령어를 사용합니다.\
행렬의 역행열은 numpy가 제공하는 .linalg.inv(X)명령어를 사용합니다.\
transpose연산은 매트릭스 뒤에 .T를 붙이면 됩니다.

In [5]:
B = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
print(B)

[125.99726381  15.29678219  21.29186902   8.56317855   5.10219982
   6.17888509]


In [6]:
print("Intercept:", B[0])
print("coefficient of wp:", B[1])
print("coefficient of ps:", B[2])
print("coefficient of ws:", B[3])
print("coefficient of AL:", B[4])
print("coefficient of smsa:", B[5])
# print the formula
print("Revenue Function: rev = {} + {} wp + {} ps + {} ws + {} AL + {} smsa".format(B[0],B[1],B[2],B[3],B[4],B[5]))

Intercept: 125.997263812184
coefficient of wp: 15.296782190777414
coefficient of ps: 21.291869021982837
coefficient of ws: 8.56317854601312
coefficient of AL: 5.102199822482363
coefficient of smsa: 6.178885090151204
Revenue Function: rev = 125.997263812184 + 15.296782190777414 wp + 21.291869021982837 ps + 8.56317854601312 ws + 5.102199822482363 AL + 6.178885090151204 smsa


이 외에도 이를 이용해서 잔차의 Standard error를 계산하고 각각의 통계적 유의성을 구하는 과정이 남아있습니다.

**Statsmodels 패키지를 활용하기**\
Statsmodels 패키지는 간단하게 회귀분석을 할 수 있는 함수를 제공합니다. 이를 이용해서 쉽게 회귀분석을 해볼 수 있습니다.\
회귀식을 입력할 때는 '종속변수 ~ 피쳐1+피쳐2+...+피쳐n'형식으로 입력합니다.

In [26]:
#pip install statsmodels

In [7]:
import statsmodels.formula.api as smf

In [8]:
model = smf.ols('rev ~ wp + ps + ws + AL + smsa', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    rev   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     39.92
Date:                Sat, 23 Mar 2024   Prob (F-statistic):           7.03e-34
Time:                        11:08:53   Log-Likelihood:                -2384.0
No. Observations:                 450   AIC:                             4780.
Df Residuals:                     444   BIC:                             4805.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    125.9973     20.336      6.196      0.0